In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset
import numpy as np

In [3]:
data = Dataset(name='imdb', dirname='preprocess/IMDB/')

INFO - 2019-02-22 14:43:05,115 - Reading Structured data ...
INFO - 2019-02-22 14:43:05,147 - Reading Notes ...
INFO - 2019-02-22 14:43:06,010 - Stratifying ...
WARNING - 2019-02-22 14:43:06,027 - No records for dev split


In [4]:
labellist = ['sentiment']
data.generate_labels(labellist, len(labellist), 'binary')
data.save_on_metric = 'roc_auc'
data.metrics_type = 'classifier'

In [5]:
train_data = data.get_data('train', structured=False)
test_data = data.get_data('test', structured=False)

Pos Percentage [0.5]
Pos Percentage [0.5]


In [11]:
from PatientVec.models.baselines.LR import LR, LDA
from PatientVec.Experiments.hyperparam_exps import get_basic_data

train_data, dev_data = data.get_data('train', structured=False), data.get_data('test', structured=False)
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)

Pos Percentage [0.5]
Pos Percentage [0.5]



Fit BOW Classifier ...
Fit TFIDF Classifier ...



BOW
{'accuracy': 0.8674, 'roc_auc': 0.9404451199999999, 'pr_auc': 0.9385207504655958}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.867,0.868,0.867,0.867,0.867
precision,0.871,0.864,0.867,0.867,0.867
recall,0.863,0.872,0.867,0.867,0.867
support,2500.000,2500.000,5000.000,5000.000,5000.000


TFIDF
{'accuracy': 0.8588, 'roc_auc': 0.93241792, 'pr_auc': 0.930023271363641}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.858,0.860,0.859,0.859,0.859
precision,0.864,0.854,0.859,0.859,0.859
recall,0.852,0.866,0.859,0.859,0.859
support,2500.000,2500.000,5000.000,5000.000,5000.000


In [12]:
from PatientVec.models.baselines.LR import LR, LDA
from PatientVec.Experiments.hyperparam_exps import get_basic_data

train_data, dev_data = data.get_data('train', structured=False), data.get_data('test', structured=False)
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)

Pos Percentage [0.5]
Pos Percentage [0.5]



Fit BOW Classifier ...
Fit TFIDF Classifier ...



BOW
{'accuracy': 0.882, 'roc_auc': 0.94537168, 'pr_auc': 0.9429760109515799}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.881,0.883,0.882,0.882,0.882
precision,0.889,0.875,0.882,0.882,0.882
recall,0.873,0.891,0.882,0.882,0.882
support,2500.000,2500.000,5000.000,5000.000,5000.000


TFIDF
{'accuracy': 0.8848, 'roc_auc': 0.9475115199999999, 'pr_auc': 0.9449284944533765}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.884,0.886,0.885,0.885,0.885
precision,0.893,0.877,0.885,0.885,0.885
recall,0.875,0.895,0.885,0.885,0.885
support,2500.000,2500.000,5000.000,5000.000,5000.000
